# Pesquisas - considerar janela de contexto na classificação
---

## Importando Libs
---


In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F

!pip install transformers==4.30.2
!pip install simpletransformers
from simpletransformers.language_representation import RepresentationModel

In [ ]:
# Carasdasregando o modelo BERT em pofdrtuguês # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português # Carregando o modelo BERT em português

## Modelo BERT
---

In [ ]:
# Carregando o modelo BERT em português
model = RepresentationModel(
    model_type="bert",
    model_name="neuralmind/bert-base-portuguese-cased",
    use_cuda=True,  # Defina como True se tiver GPU
)

## Magia
---

### Similaridade do cosseno

In [ ]:
def calculate_cosine_distance(vec1, vec2):
    vec1_tensor = torch.tensor(vec1)
    vec2_tensor = torch.tensor(vec2)

    # Calculando a similaridade do cosseno
    cosine_similarity = F.cosine_similarity(vec1_tensor, vec2_tensor).item()
    return cosine_similarity

### Cálculo de distância
---

In [ ]:
def magia(prev_question, current_question, threshold=0.7):
    # Obtenha os embeddings das perguntas usando BERT
    prev_embedding = model.encode_sentences([prev_question], combine_strategy="mean")  # Anterior
    current_embedding = model.encode_sentences([current_question], combine_strategy="mean")  # Atual

    # Calcule a distância do cosseno
    cosine_distance = calculate_cosine_distance(prev_embedding, current_embedding)
    # print("Esse é meu threshold: ", cosine_distance)

    # Verifique se a pergunta é de seguimento com base no limiar
    if cosine_distance > threshold:
        return "Pergunta Seguimentar"
    else:
        return "Pergunta Base"

### Testes aleatórios
---

In [ ]:
# Teste
prev_question = "Alguma vez você foi diagnosticado com alguma condição médica crônica?"  # Anterior
current_question = "Te passaram qual medicamento?"  # Atual
print(magia(prev_question, current_question))

Pergunta Base


## Avaliação | Dados de validação

In [ ]:
df = pd.read_csv("validacao.csv")
df.head(10)

,Pergunta,Label
0,Você faz uso de algum medicamento de forma reg...,Pergunta Base
1,Há quanto tempo você começou a tomar esse medi...,Pergunta Seguimentar
2,Existem suplementos ou vitaminas que você este...,Pergunta Base
3,Um médico ou profissional de saúde recomendou ...,Pergunta Seguimentar
4,Houve alguma mudança em sua medicação nos últi...,Pergunta Base
5,Qual foi a razão por trás dessa alteração na m...,Pergunta Seguimentar
6,Você já teve alguma reação alérgica a algum me...,Pergunta Base
7,Que tipo de reação você teve e como isso foi t...,Pergunta Seguimentar
8,Você tem alguma preocupação em relação à sua m...,Pergunta Base
9,Que tipo de preocupações você tem e como você ...,Pergunta Seguimentar


### Caminho Feliz
---
Classificação somente das perguntas seguimentares

In [ ]:
# Caminho feliz
# Lista para armazenar as previsões
predictions = []

# Iterando sobre as perguntas de dois em dois
for i in range(0, len(df), 2):
    # Se houver uma pergunta seguimentar após a pergunta base
    if i + 1 < len(df):
        prev_question = df.iloc[i]["Pergunta"]
        current_question = df.iloc[i + 1]["Pergunta"]

        # Obtenha a previsão para a pergunta base (sempre será 'Pergunta Base')
        predictions.append("Pergunta Base")

        # Obtenha a previsão para a pergunta seguimentar usando a função magia
        prediction = magia(prev_question, current_question)
        predictions.append(prediction)
    else:
        # Se não houver pergunta seguimentar após a pergunta base, apenas adicione a previsão da pergunta base
        predictions.append("Pergunta Base")

# Adicionando a coluna de previsão ao dataframe
df["Previsão"] = predictions

accuracy = (df["Label"] == df["Previsão"]).mean()
accuracy

0.88

### Classificação do dataset todo
---
A primeira linha é definida como 'Pergunta Base'

In [ ]:
# Percorrendo a planilha
# Lista para armazenar as previsões
predictions = []

# Adiciona a previsão para a primeira pergunta (sempre será 'Pergunta Base')
predictions.append("Pergunta Base")

# Iterando a partir da segunda pergunta
for i in range(1, len(df)):
    prev_question = df.iloc[i - 1]["Pergunta"]
    current_question = df.iloc[i]["Pergunta"]

    # Obtenha a previsão usando a função magia
    prediction = magia(prev_question, current_question)
    predictions.append(prediction)

# Adicionando a coluna de previsão ao dataframe
df["Previsão"] = predictions

accuracy = (df["Label"] == df["Previsão"]).mean()
accuracy

0.74

### Exportar resultados

In [ ]:
# df.to_csv('/content/validacao_atualizado.csv', index=False)

## Avaliação 1 | Simplício
---
Olhando 1 pergunta anterior

Importação dos dados

In [ ]:
df = pd.read_csv("Simplicio.csv")
df.head(10)

,Pergunta,Label
0,"Oi, Simplício, tudo bem?",Pergunta Base
1,Quais os seus sintomas?,Pergunta Base
2,Desde quando você tem esses sintomas?,Pergunta Seguimentar
3,Você foi diagnosticado do quê?,Pergunta Base
4,"Me conte desses oito anos para cá, o que acont...",Pergunta Base
5,Como tem sido seu tratamento para a diabetes?,Pergunta Base
6,Você toma qual comprimido?,Pergunta Seguimentar
7,Quando foi a última vez que você fez a dosagem...,Pergunta Base
8,E você tinha mais algum sintoma?,Pergunta Seguimentar
9,Como é o seu sono?,Pergunta Base


Percorrendo a planilha do Simplicio:

In [ ]:
# Lista para armazenar as previsões
predictions = []

# Adiciona a previsão para a primeira pergunta (sempre será 'Pergunta Base')
predictions.append("Pergunta Base")

# Iterando a partir da segunda pergunta
for i in range(1, len(df)):
    prev_question = df.iloc[i - 1]["Pergunta"]
    current_question = df.iloc[i]["Pergunta"]

    # Obtenha a previsão usando a função magia
    prediction = magia(prev_question, current_question)
    predictions.append(prediction)

# Adicionando a coluna de previsão ao dataframe
df["Previsão"] = predictions

accuracy = (df["Label"] == df["Previsão"]).mean()
accuracy

0.7142857142857143

### Exportar resultados

In [ ]:
df.to_csv("/content/validacao_Simplício_atualizado.csv", index=False)

## Avaliação 2 | Simplício
---
Olhando 2 perguntas anteriores

In [ ]:
def magia_expandida(prev_question1, prev_question2, current_question, threshold=0.7):
    # Obtenha os embeddings das perguntas usando BERT
    prev_embedding1 = model.encode_sentences([prev_question1], combine_strategy="mean")  # Anterior 1
    prev_embedding2 = model.encode_sentences([prev_question2], combine_strategy="mean")  # Anterior 2
    current_embedding = model.encode_sentences([current_question], combine_strategy="mean")  # Atual

    # Calcule a distância do cosseno para ambas as perguntas anteriores
    cosine_distance1 = calculate_cosine_distance(prev_embedding1, current_embedding)
    cosine_distance2 = calculate_cosine_distance(prev_embedding2, current_embedding)

    # Decisão com base nas duas distâncias
    # No exemplo abaixo, considera-se "Pergunta Seguimentar" se pelo menos uma das distâncias exceder o limiar.
    if cosine_distance1 > threshold or cosine_distance2 > threshold:
        return "Pergunta Seguimentar"
    else:
        return "Pergunta Base"


# Percorrendo a planilha
predictions = []

# Adiciona a previsão para as duas primeiras perguntas (elas serão sempre 'Pergunta Base' neste contexto)
predictions.extend(["Pergunta Base", "Pergunta Base"])

# Iterando a partir da terceira pergunta
for i in range(2, len(df)):
    prev_question1 = df.iloc[i - 2]["Pergunta"]
    prev_question2 = df.iloc[i - 1]["Pergunta"]
    current_question = df.iloc[i]["Pergunta"]

    # Obtenha a previsão usando a função magia_expandida
    prediction = magia_expandida(prev_question1, prev_question2, current_question)
    predictions.append(prediction)

# Adicionando a coluna de previsão ao dataframe
df["Previsão"] = predictions

accuracy = (df["Label"] == df["Previsão"]).mean()
accuracy

0.7428571428571429